Importation des packages :

In [8]:
import pickle
import pandas as pd
from random import randint
import re
import unicodedata 
import time
import pandas as pd
import numpy as np
import random
import nltk
import collections
import csv
import pickle
from pandas import DataFrame
from sklearn.feature_extraction.text import TfidfVectorizer # ou CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Importation des vectoriseurs, de la fonction de normalisation et du dataFrame normalisé : 

In [9]:
dtm_question = pd.read_pickle('dtm_question.pk')
dtm_reponse = pd.read_pickle('dtm_reponse.pk')

nltk_stopwords = nltk.corpus.stopwords.words('french')
stopwords = list(nltk_stopwords)
stemmer=nltk.stem.SnowballStemmer('french')

#data 
data = pd.read_csv('dataQR.csv', sep=';')

#data normalisée 
data_norm = pd.read_csv('dataQR_normalize.csv', sep=';')

def nettoyage_texte(txt):
    #on met tout le texte en minuscule, et on remplce les retours à la ligne par des espaces
    txt = txt.lower().replace('\n', ' ')

    #on retire tous les accents 
    txt = unicodedata.normalize('NFD', txt).encode('ascii', 'ignore')

    #on garde uniquement les caractères alphanumériques : 
    txt = re.sub('[^a-z_]', ' ', str(txt))
    
    #on retire les stops words
    tokens = [w for w in txt.split() if (len(w)>2) and (w not in nltk_stopwords)]
    
    #pour avoir les mots racines
    tokens = [stemmer.stem(token) for token in tokens]

    return ' '.join(tokens)

In [10]:
def similarity(texte) : 
    #liste des valeurs des similarités 
    similarity = []
    
    #liste des valeurs des index 
    index_similarity = []
    
    
    #on teste la similarité avec les questions de notre base Vinted 
    dtm_q = dtm_question.transform(data_norm['Question_normalize'])
    req_vect_q = dtm_question.transform(pd.Series([nettoyage_texte(texte)]))
    
    query_corpus_sim_quest = np.squeeze(cosine_similarity(dtm_q, req_vect_q))
    
    #on prend la plus grande similarité (attention si plusieurs)
    sim_quest = np.max(query_corpus_sim_quest)
    #on l'ajoute à la liste 
    similarity.append(sim_quest)
    
    #index de la valeur la plus grande
    sim_quest_ind = np.argmax(query_corpus_sim_quest)
    #on l'ajoute à la liste 
    index_similarity.append(sim_quest_ind)
    
    #on teste la similarité avec les réponses de notre base Vinted 
    dtm_r = dtm_reponse.transform(data_norm['Reponse_normalize'])
    req_vect_r = dtm_reponse.transform(pd.Series([nettoyage_texte(texte)]))
    
    query_corpus_sim_rep = np.squeeze(cosine_similarity(dtm_r, req_vect_r))
    
    sim_rep = np.max(query_corpus_sim_rep)
    similarity.append(sim_rep)
    
    sim_rep_ind = np.argmax(query_corpus_sim_rep)
    index_similarity.append(sim_rep_ind)
    
    if max(similarity) > 0.50 : 
        return data.iloc[index_similarity[similarity.index(max(similarity))],1] + '\n'
    else : 
        return 'autre_question'

In [28]:
def Chatbot() : 
    #on récupère la question posée par l'utilisateur 
    phrase_user = input("-Chatbot : \nSalut, comment puis-je t'aider ? \n-User : \n\n")
    
    #boucle while permettant la continuité du dialogue
    #si l'utilisateur nous dit au revoir alors le programme s'arrête 
    while not re.search(r"(.*)au revoir", phrase_user, re.IGNORECASE):
        #on normalise la phrase entrée par l'utilisateur 
        phrase_norm = nettoyage_texte(phrase_user)
        
        if similarity(phrase_norm) == 'autre_question' : 
            phrase_user = input("\n" + "-Chatbot :" + "\n" + "=> Réponse : "  + "\nPose une autre question \n-User : \n\n")
        else : 
            phrase_user = input("\n" + "=> Réponse : " + "\n" + similarity(phrase_norm) + '\n' + "As-tu d'autres questions ? \n-User : \n")

In [29]:
Chatbot()

-Chatbot : 
Salut, comment puis-je t'aider ? 
-User : 

 je n'arrive pas à me connecter à mon compte

=> Réponse : 
Tu n’arrives pas à te connecter à ton compte ?
Essaie de te connecter avec ton adresse e-mail à la place de ton identifiant (utilise l’adresse e-mail utilisée lors de ton inscription sur Vinted) ou ton compte Facebook (si tu t’es inscrit sur Vinted de cette façon).Assure-toi de bien te connecter sur vinted.fr ou sur l’appli de Vinted France.Tu as oublié ton mot de passe ?
Clique surSe connecter et surMot de passe oubliéEntre ton adresse e-mail et clique surValiderClique sur le boutonNouveau mot de passe que tu peux trouver dans le mail envoyé par l’équipe Vinted.Tu pourras ensuite créer un nouveau mot de passe et de nouveau accéder à ton compte Vinted.
Tu n’arrives toujours pas à te connecter ? Ton compte est peut-être suspendu :
Tu dois peut-êtrefournir une pièce d’identité à l’équipe Vinted.Tu dois peut-être faire vérifier ton compte avec ton numéro de téléphone.Ton com